In [1]:
import akshare as ak
import pandas as pd
import os
import baostock as bs

In [2]:
bs.login()

login success!


In [3]:
fund_etf_category_sina_df = ak.fund_etf_category_sina(symbol="ETF基金")

In [4]:
fund_etf_category_sina_df['qmt_code'] = fund_etf_category_sina_df['代码'].str[2:] + '.' + fund_etf_category_sina_df['代码'].str[:2].str.upper()

In [5]:
fund_etf_category_sina_df.to_csv('data/fund_etf_category_sina.csv')

In [6]:
fund_etf_category_sina_df

,代码,名称,最新价,涨跌额,涨跌幅,买入,卖出,昨收,今开,最高,最低,成交量,成交额,qmt_code
0,sz159998,计算机ETF,1.017,-0.005,-0.489,1.016,1.017,1.022,1.003,1.020,1.003,99484300,100706519,159998.SZ
1,sz159997,电子ETF,1.211,-0.009,-0.738,1.211,1.213,1.220,1.203,1.219,1.199,65247204,78518815,159997.SZ
2,sz159996,家电ETF,1.391,-0.007,-0.501,1.391,1.392,1.398,1.375,1.393,1.371,47410078,65548807,159996.SZ
3,sz159995,芯片ETF,1.346,-0.011,-0.811,1.346,1.347,1.357,1.337,1.354,1.331,600150620,804402212,159995.SZ
4,sz159994,5GETF,0.956,-0.004,-0.417,0.955,0.956,0.960,0.949,0.960,0.942,29174105,27757639,159994.SZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,sh510050,上证50ETF,2.739,0.012,0.440,2.739,2.740,2.727,2.710,2.740,2.705,616194298,1676905021,510050.SH
1093,sh510040,上证180ETF指数,1.000,0.001,0.100,0.993,1.000,0.999,0.993,1.000,0.991,4190500,4169770,510040.SH
1094,sh510030,价值ETF,0.981,0.003,0.307,0.977,0.982,0.978,0.971,0.982,0.970,11029200,10752696,510030.SH
1095,sh510020,超大盘ETF,3.094,-0.001,-0.032,3.093,3.099,3.095,3.085,3.095,3.072,154100,474893,510020.SH


In [7]:
etf_sina_savepath = 'etf_sina'
if not os.path.exists(etf_sina_savepath):
    os.mkdir(etf_sina_savepath)

In [8]:
etf_codes = fund_etf_category_sina_df['代码'].to_list()

In [54]:

# for fund_code in etf_codes:
#     try:
#         df = ak.fund_etf_hist_sina(fund_code)
#         df['pctChg'] = df['close'].pct_change().fillna(0)
#         df['code'] = fund_code
#         df = df[['date', 'code', 'open', 'high', 'low', 'close', 'volume', 'pctChg']]
#         df.to_csv(f'{etf_sina_savepath}/{fund_code}.csv', index=False)
#     except:
#         print(fund_code)
#         continue

In [9]:
def get_fund_etf_hist_sina(code):
    try:
        return pd.read_csv(f"{etf_sina_savepath}/{code}.csv")
    except:
        try:
            df = ak.fund_etf_hist_sina(code)
            df['pctChg'] = df['close'].pct_change().fillna(0)
            df['code'] = code
            df = df[['date', 'code', 'open', 'high', 'low', 'close', 'volume', 'pctChg']]
            df.to_csv(f'{etf_sina_savepath}/{code}.csv', index=False)
            return df
        except:
            print(f"{code} not found")

In [10]:
fund_etf_category_sina_returns = {}
for code in etf_codes:
    try:
        fund_etf_category_sina_returns[code] = get_fund_etf_hist_sina(code)['pctChg']
    except:
        print(code)
        continue

In [11]:
returns_df = pd.DataFrame(fund_etf_category_sina_returns).dropna(axis=1, how='all')

In [12]:
corr_matrix = returns_df.corr(method='pearson')

In [13]:
threshold = 0.8  # 相似度阈值
to_drop = set()

for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if corr_matrix.iloc[i, j] > threshold:
            # 保留规模较大的基金
            fund_i = corr_matrix.columns[i]
            fund_j = corr_matrix.columns[j]
            if fund_etf_category_sina_df.loc[fund_etf_category_sina_df['代码'] == fund_i, '成交额'].values[0] \
                > fund_etf_category_sina_df.loc[fund_etf_category_sina_df['代码'] == fund_j, '成交额'].values[0]:
                to_drop.add(fund_j)
            else:
                to_drop.add(fund_i)


In [14]:
len(to_drop)

779

In [15]:
filtered_codes = [code for code in returns_df.columns if code not in to_drop]

In [16]:
len(filtered_codes)

318

In [17]:
final_df = fund_etf_category_sina_df[fund_etf_category_sina_df['代码'].isin(filtered_codes)]
final_df = final_df.sort_values(by='成交额', ascending=False).head(300)  # 取前300名


In [18]:
final_df

,代码,名称,最新价,涨跌额,涨跌幅,买入,卖出,昨收,今开,最高,最低,成交量,成交额,qmt_code
1038,sh511090,30年国债ETF,121.002,-1.257,-1.028,121.000,121.001,122.259,122.442,122.500,121.000,86611406,10518708697,511090.SH
1000,sh511990,华宝添益ETF,100.002,-0.003,-0.003,100.001,100.002,100.005,100.003,100.005,100.000,94901632,9490364364,511990.SH
919,sh513180,恒生科技指数ETF,0.812,0.002,0.247,0.812,0.813,0.810,0.794,0.819,0.790,11543926800,9308959343,513180.SH
924,sh513130,恒生科技ETF,0.795,0.002,0.252,0.794,0.795,0.793,0.776,0.802,0.771,11606317570,9165030307,513130.SH
1009,sh511880,银华日利ETF,100.305,0.007,0.007,100.305,100.306,100.298,100.302,100.305,100.301,77467750,7770239540,511880.SH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,sz159736,饮食ETF,0.723,0.013,1.831,0.723,0.724,0.710,0.707,0.724,0.704,31928301,22854777,159736.SZ
596,sh561560,电力ETF,1.111,0.002,0.180,1.111,1.112,1.109,1.106,1.112,1.102,20097700,22262226,561560.SH
577,sh562000,中证A100ETF基金,0.964,0.000,0.000,0.963,0.965,0.964,0.961,0.964,0.955,23141000,22185875,562000.SH
706,sh517800,人工智能50ETF,0.841,-0.014,-1.637,0.841,0.842,0.855,0.840,0.848,0.833,26410200,22155332,517800.SH


In [19]:
final_df[['代码', '名称', 'qmt_code']].to_csv('data/fund_etf_top300_sina.csv', index=False)